# Chapter 14: Introduction to LangGraph
**From: Zero to AI Agent**

## Overview
In this chapter, you'll learn about:
- Why LangGraph? Limitations of simple chains
- Graph-based agent architectures
- Installing and setting up LangGraph
- Core concepts: nodes, edges, and state
- Your first LangGraph application
- Conditional edges and branching logic
- Debugging LangGraph flows


In [ ]:
!pip install -q -r requirements.txt

from dotenv import load_dotenv
load_dotenv()

---
## Section 14.1: Why LangGraph? Limitations of simple chains

In [ ]:
# Section 14.1 content
# No source files found for this section

---
### Section 14.1 Exercises

### Exercise 14.1.1: Identify Chain Limitations

Think of an AI application you'd like to build (or pick one: email assistant, study tutor, recipe suggester). Write down:

1. What steps would it need to perform?
2. Where might it need to loop back (retry or refine)?
3. Where might it need to branch (handle different cases)?
4. What information would it need to track across steps?

In [ ]:
# Your code here


### Exercise 14.1.2: Flowchart Design

Draw a flowchart (on paper or describe it in text) for a "Smart Email Responder" that:

- Reads an incoming email
- Classifies it (urgent, normal, spam)
- For urgent: drafts an immediate response
- For normal: adds to a queue
- For spam: archives it
- For drafted responses: gets human approval
- If human requests changes: loops back to redraft

Identify which parts would be impossible or messy with a simple chain.

In [ ]:
# Your code here


### Exercise 14.1.3: Analyze the Pattern

Look at this pseudo-code:

```python
def smart_assistant(task):
    plan = create_plan(task)
    
    while not is_complete(plan):
        next_step = get_next_step(plan)
        result = execute_step(next_step)
        
        if result.failed:
            if result.retryable:
                continue  # Try same step again
            else:
                plan = revise_plan(plan, result.error)
        else:
            update_plan(plan, result)
    
    return summarize_results(plan)
```

Explain why this would be difficult with simple chains. Identify: the loops, the branching points, and what state needs to persist.

In [ ]:
# Your code here


---
## Section 14.2: Graph-based agent architectures

In [ ]:
# Section 14.2 content
# No source files found for this section

---
### Section 14.2 Exercises

### Exercise 14.2.1: Pattern Recognition

For each scenario, identify which pattern(s) would be most appropriate:

1. An agent that translates a document from English to Spanish
2. An agent that keeps asking clarifying questions until it understands the user's request
3. An agent that checks the weather in three cities simultaneously
4. An agent that writes code, runs tests, and fixes bugs until all tests pass
5. An agent that drafts a legal contract and requires lawyer approval before finalizing

In [ ]:
# Your code here


### Exercise 14.2.2: Design a Recipe Agent

Design a graph for a cooking assistant agent that:
- Takes a dish the user wants to make
- Checks what ingredients the user has available
- Finds a suitable recipe (might need to search multiple times for alternatives)
- Adjusts the recipe based on available ingredients
- Generates step-by-step cooking instructions
- Can answer questions during cooking (loops back to handle questions)

Sketch the graph and identify: the nodes, the decision points, any loops, and what state you'd need.

In [ ]:
# Your code here


### Exercise 14.2.3: Identify the State

For the customer service agent we designed in this section, list all the pieces of information that should be in the state. For each piece, explain which node(s) would write to it and which node(s) would read from it.

In [ ]:
# Your code here


---
## Section 14.3: Installing and setting up LangGraph

In [ ]:
# From: verify_install.py

# From: Building AI Agents, Chapter 14, Section 14.3
# File: verify_install.py

"""Verify that LangGraph is installed correctly."""

def check_installation():
    """Check all required packages."""
    print("🔍 Checking LangGraph installation...\n")
    
    # Check langgraph
    try:
        import langgraph
        print(f"✅ langgraph installed (version: {langgraph.__version__})")
    except ImportError:
        print("❌ langgraph not installed")
        return False
    except AttributeError:
        print("✅ langgraph installed (version not available)")
    
    # Check langchain
    try:
        import langchain
        print(f"✅ langchain installed (version: {langchain.__version__})")
    except ImportError:
        print("❌ langchain not installed")
        return False
    
    # Check langchain-openai
    try:
        from langchain_openai import ChatOpenAI
        print("✅ langchain-openai installed")
    except ImportError:
        print("❌ langchain-openai not installed")
        return False
    
    # Check python-dotenv
    try:
        import dotenv
        print("✅ python-dotenv installed")
    except ImportError:
        print("❌ python-dotenv not installed")
        return False
    
    print("\n🎉 All packages installed correctly!")
    return True

if __name__ == "__main__":
    check_installation()


In [ ]:
# From: verify_api.py

# From: Building AI Agents, Chapter 14, Section 14.3
# File: verify_api.py

"""Verify API connection is working."""

import os
from dotenv import load_dotenv

def check_api():
    """Check that we can connect to OpenAI."""
    print("🔍 Checking API connection...\n")
    
    # Load environment variables
    load_dotenv()
    
    # Check for API key
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("❌ OPENAI_API_KEY not found in environment")
        print("   Make sure you have a .env file with your API key")
        return False
    
    if not api_key.startswith("sk-"):
        print("⚠️  API key doesn't start with 'sk-' - it might be invalid")
    
    print("✅ API key found")
    
    # Try to make a simple API call
    print("\n🔄 Testing API connection...")
    
    try:
        from langchain_openai import ChatOpenAI
        
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
        response = llm.invoke("Say 'Hello, LangGraph!' and nothing else.")
        
        print(f"✅ API connection successful!")
        print(f"   Response: {response.content}")
        return True
        
    except Exception as e:
        print(f"❌ API connection failed: {e}")
        return False

if __name__ == "__main__":
    check_api()


In [ ]:
# From: verify_langgraph.py

# From: Building AI Agents, Chapter 14, Section 14.3
# File: verify_langgraph.py

"""Verify LangGraph components are accessible."""

def check_langgraph():
    """Check that we can import LangGraph components."""
    print("🔍 Checking LangGraph components...\n")
    
    try:
        # Core graph components
        from langgraph.graph import StateGraph, END
        print("✅ StateGraph imported (for building graphs)")
        print("✅ END imported (for marking end states)")
        
        # State management
        from typing import TypedDict
        print("✅ TypedDict available (for defining state)")
        
        # Checkpointing (for persistence)
        from langgraph.checkpoint.memory import MemorySaver
        print("✅ MemorySaver imported (for state persistence)")
        
        print("\n🎉 All LangGraph components ready!")
        print("\nYou can now build graphs with:")
        print("  - StateGraph: Define your graph structure")
        print("  - Nodes: Add processing steps")
        print("  - Edges: Connect steps together")
        print("  - State: Share data between nodes")
        
        return True
        
    except ImportError as e:
        print(f"❌ Import failed: {e}")
        return False

if __name__ == "__main__":
    check_langgraph()


---
### Section 14.3 Exercises

### Exercise 14.3.1: Environment Exploration

Run `pip list` in your terminal and find all the packages that were installed as dependencies of LangGraph. Count how many there are. Then look up what three of them do (pick ones with interesting names).

In [ ]:
# Your code here


### Exercise 14.3.2: API Key Security

Explain in your own words why we use a `.env` file instead of putting the API key directly in our code. What could go wrong if you accidentally committed an API key to a public GitHub repository?

In [ ]:
# Your code here


### Exercise 14.3.3: Create a Setup Checker

Combine all three verification scripts into one comprehensive `setup_check.py` that:
- Checks all package installations
- Verifies the API key exists and has the right format
- Tests the API connection
- Reports a summary at the end with overall pass/fail

Make it user-friendly with clear instructions if anything fails.

In [ ]:
# Your code here


---
## Section 14.4: Core concepts: nodes, edges, and state

In [ ]:
# From: writer_loop.py

# From: Building AI Agents, Chapter 14, Section 14.4
# File: writer_loop.py

"""
A simple feedback loop demonstrating LangGraph core concepts:
- State with TypedDict
- Nodes that read/write state
- Conditional edges for looping
- The add reducer for list accumulation
"""

from typing import TypedDict, Annotated
from operator import add
from langgraph.graph import StateGraph, END


# 1. Define our state
class WriterState(TypedDict):
    topic: str                           # What to write about
    drafts: Annotated[list, add]         # Accumulate drafts
    current_draft: str                   # Latest draft
    quality_score: int                   # How good is it (1-10)


# 2. Define our nodes
def write_draft(state: WriterState) -> dict:
    """Write or rewrite a draft."""
    topic = state["topic"]
    attempt = len(state.get("drafts", [])) + 1
    
    # In reality, this would call an LLM
    draft = f"Draft {attempt} about {topic}: [content here]"
    
    return {
        "current_draft": draft,
        "drafts": [draft]  # Appends due to Annotated[list, add]
    }


def evaluate_draft(state: WriterState) -> dict:
    """Score the current draft."""
    draft = state["current_draft"]
    
    # In reality, this would use an LLM or other logic
    # For demo, score increases with each attempt
    score = min(len(state.get("drafts", [])) * 3, 10)
    
    return {"quality_score": score}


def decide_if_done(state: WriterState) -> str:
    """Decide whether to finish or revise."""
    if state["quality_score"] >= 7:
        return "done"
    elif len(state.get("drafts", [])) >= 3:
        return "done"  # Give up after 3 attempts
    else:
        return "revise"


# 3. Build the graph
graph = StateGraph(WriterState)

# Add nodes
graph.add_node("write", write_draft)
graph.add_node("evaluate", evaluate_draft)

# Add edges
graph.set_entry_point("write")
graph.add_edge("write", "evaluate")
graph.add_conditional_edges(
    "evaluate",
    decide_if_done,
    {
        "done": END,
        "revise": "write"  # Loop back!
    }
)

# 4. Compile the graph
app = graph.compile()

# 5. Run it!
if __name__ == "__main__":
    result = app.invoke({"topic": "AI agents", "drafts": []})
    print(f"Final draft: {result['current_draft']}")
    print(f"Total attempts: {len(result['drafts'])}")
    print(f"Final score: {result['quality_score']}")
    
    # Show all drafts
    print("\nAll drafts:")
    for i, draft in enumerate(result['drafts'], 1):
        print(f"  {i}. {draft}")


---
### Section 14.4 Exercises

### Exercise 14.4.1: Design a State

Design the state TypedDict for a "Code Review Agent" that:
- Receives code to review
- Identifies issues (could be multiple)
- Suggests fixes for each issue
- Tracks which issues have been addressed
- Knows when the review is complete

Think about: What fields do you need? Which should be lists? Which need the `add` reducer?

In [ ]:
# Your code here


### Exercise 14.4.2: Write the Nodes

Using the state you designed in Exercise 1, write pseudocode (or real code) for three nodes:
- `analyze_code`: Looks at the code and identifies issues
- `suggest_fix`: Takes one issue and suggests a fix
- `check_complete`: Determines if all issues are addressed

Focus on: What does each node read from state? What does it write back?

In [ ]:
# Your code here


### Exercise 14.4.3: Draw the Graph

Sketch the graph (on paper or ASCII art) for the Code Review Agent. Include:
- Where it starts
- The flow between nodes
- Any conditional edges (what are the conditions?)
- Where loops occur
- Where it ends

Then write the LangGraph code to build this graph structure (just the graph building part—nodes can be placeholder functions).


This complete solution includes the state design (Exercise 1), node implementations (Exercise 2), and graph construction (Exercise 3) all in one runnable file.

In [ ]:
# Your code here


---
## Section 14.5: Your first LangGraph application

In [ ]:
# From: self_improving_writer.py

# From: Building AI Agents, Chapter 14, Section 14.5
# File: self_improving_writer.py

"""A LangGraph application that writes and improves content iteratively.

This demonstrates the fundamental generate → evaluate → improve → repeat pattern
used in many AI agents.
"""

import os
from typing import TypedDict
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

# Load environment variables
load_dotenv()

# Initialize our LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)


# === STATE ===

class WriterState(TypedDict):
    topic: str                    # The topic to write about
    draft: str                    # Current draft
    critique: str                 # Feedback on the draft
    revision_count: int           # How many revisions so far
    max_revisions: int            # Maximum revisions allowed


# === NODES ===

def write(state: WriterState) -> dict:
    """Write or revise the draft based on current state."""
    topic = state["topic"]
    draft = state.get("draft", "")
    critique = state.get("critique", "")
    revision_count = state.get("revision_count", 0)

    if not draft:
        # Initial draft - no existing content
        prompt = f"""Write a short, informative paragraph about: {topic}

        Keep it concise but engaging. Aim for 3-4 sentences."""

        response = llm.invoke(prompt)
        print(f"📝 Initial draft written ({len(response.content)} chars)")

        return {
            "draft": response.content,
            "revision_count": 0
        }
    else:
        # Revision - improve based on critique
        prompt = f"""Revise this draft about "{topic}" based on the feedback provided.

        Current draft:
        {draft}

        Feedback:
        {critique}

        Write an improved version that addresses the feedback. Keep it concise."""

        response = llm.invoke(prompt)
        new_count = revision_count + 1
        print(f"✏️ Revision {new_count} complete")

        return {
            "draft": response.content,
            "revision_count": new_count
        }


def critique_draft(state: WriterState) -> dict:
    """Analyze the draft and provide constructive feedback."""
    draft = state["draft"]
    topic = state["topic"]

    prompt = f"""Review this draft about "{topic}" and provide brief, constructive feedback.

    Draft:
    {draft}

    Focus on:
    1. Is the information accurate and complete?
    2. Is it engaging and well-written?
    3. What specific improvements would make it better?

    If the draft is already excellent, say "EXCELLENT" at the start of your response.
    Otherwise, provide 2-3 specific suggestions for improvement."""

    response = llm.invoke(prompt)

    print(f"🔍 Critique: {response.content[:100]}...")

    return {"critique": response.content}


# === DECISION FUNCTION ===

def should_continue(state: WriterState) -> str:
    """Decide whether to revise again or finish."""
    critique = state["critique"]
    revision_count = state["revision_count"]
    max_revisions = state["max_revisions"]
    
    # Stop if we've hit the revision limit
    if revision_count >= max_revisions:
        print(f"🛑 Max revisions ({max_revisions}) reached")
        return "end"
    
    # Stop if the critique says it's excellent
    if "EXCELLENT" in critique.upper():
        print("✨ Draft deemed excellent!")
        return "end"
    
    # Otherwise, keep improving
    print("🔄 Continuing to revise...")
    return "continue"


# === GRAPH BUILDER ===

def create_writer_graph():
    """Build and return the writer graph."""

    # Create the graph with our state type
    graph = StateGraph(WriterState)

    # Add our nodes
    graph.add_node("write", write)
    graph.add_node("critique", critique_draft)

    # Set the entry point
    graph.set_entry_point("write")

    # Add edges
    graph.add_edge("write", "critique")

    graph.add_conditional_edges(
        "critique",
        should_continue,
        {
            "continue": "write",  # Loop back to write for revision
            "end": END
        }
    )

    return graph.compile()


# === MAIN ===

def main():
    """Run the self-improving writer."""
    print("=" * 50)
    print("🚀 Self-Improving Writer")
    print("=" * 50)
    
    # Create the graph
    app = create_writer_graph()
    
    # Define our initial state
    initial_state = {
        "topic": "Why learning to code is valuable in 2024",
        "draft": "",
        "critique": "",
        "revision_count": 0,
        "max_revisions": 3
    }
    
    print(f"\n📌 Topic: {initial_state['topic']}")
    print(f"📌 Max revisions: {initial_state['max_revisions']}")
    print("\n" + "-" * 50 + "\n")
    
    # Run the graph
    result = app.invoke(initial_state)
    
    # Show the final result
    print("\n" + "=" * 50)
    print("📄 FINAL DRAFT:")
    print("=" * 50)
    print(result["draft"])
    print("\n" + "-" * 50)
    print(f"Total revisions: {result['revision_count']}")


if __name__ == "__main__":
    main()


---
### Section 14.5 Exercises

### Exercise 14.5.1: Add Draft History

Modify the writer to keep a history of all drafts, not just the current one. You'll need to:
- Change the state to use `Annotated[list, add]` for drafts
- Update nodes to append drafts rather than replace
- Display all versions at the end

This lets you see how the writing evolved through revisions.

In [ ]:
# Your code here


### Exercise 14.5.2: Quality Scoring

Add a numeric quality score (1-10) to the process:
- Add a `quality_score` field to state
- Modify `critique_draft` to also output a score
- Update `should_continue` to use the score (stop when score \>= 8)
- Display the score progression at the end

In [ ]:
# Your code here


### Exercise 14.5.3: Different Writing Styles

Add a `style` parameter that changes how the writer works:
- "formal": Professional, business-like tone
- "casual": Friendly, conversational tone  
- "creative": Artistic, expressive tone

Modify the prompts in each node to respect the chosen style. Test with the same topic but different styles.

In [ ]:
# Your code here


---
## Section 14.6: Conditional edges and branching logic

In [ ]:
# From: ticket_router.py

# From: Building AI Agents, Chapter 14, Section 14.6
# File: ticket_router.py

"""A support ticket router with multi-way branching.

Demonstrates Pattern 1: Multi-Way Branching
- Classification node analyzes ticket
- Routing function decides destination (5 options)
- Specialized handlers for each category
"""

import os
from typing import TypedDict
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


# === STATE ===

class TicketState(TypedDict):
    ticket_text: str          # The customer's message
    category: str             # Classified category (BILLING, TECHNICAL, etc.)
    priority: str             # Urgency level (HIGH, MEDIUM, LOW)
    response: str             # Generated response
    needs_human: bool         # Flag for escalation


# === CLASSIFICATION NODE ===

def classify_ticket(state: TicketState) -> dict:
    """Classify the ticket into a category and priority level.
    
    This node uses the LLM to analyze the ticket text and determine:
    1. What type of issue it is (billing, technical, account, general)
    2. How urgent it is (high, medium, low)
    
    High priority tickets will be escalated regardless of category.
    """
    ticket = state["ticket_text"]
    
    prompt = f"""Classify this support ticket into exactly one category.
    
    Ticket: {ticket}
    
    Categories:
    - BILLING: Payment issues, invoices, refunds, subscriptions
    - TECHNICAL: Bugs, errors, how-to questions, feature requests
    - ACCOUNT: Login issues, password reset, profile changes
    - GENERAL: Everything else
    
    Also determine priority:
    - HIGH: Customer is angry, service is down, money involved
    - MEDIUM: Normal requests, minor issues
    - LOW: Questions, feedback, suggestions
    
    Respond in format:
    CATEGORY: <category>
    PRIORITY: <priority>"""
    
    response = llm.invoke(prompt)
    content = response.content.upper()
    
    # Parse category from response - default to GENERAL if not found
    category = "GENERAL"
    for cat in ["BILLING", "TECHNICAL", "ACCOUNT"]:
        if cat in content:
            category = cat
            break
    
    # Parse priority from response - default to MEDIUM if not found
    priority = "MEDIUM"
    for pri in ["HIGH", "LOW"]:
        if pri in content:
            priority = pri
            break
    
    print(f"📋 Classified: {category} ({priority} priority)")
    
    return {
        "category": category,
        "priority": priority
    }


# === ROUTING FUNCTION ===

def route_by_category(state: TicketState) -> str:
    """Decide which handler should process this ticket.
    
    The routing logic:
    1. HIGH priority tickets always go to escalation (human needed)
    2. Otherwise, route to the specialized handler for that category
    
    Returns a string that matches one of our handler node names.
    """
    category = state["category"]
    priority = state["priority"]
    
    # High priority always escalates, regardless of category
    if priority == "HIGH":
        return "escalate"
    
    # Map categories to handler names
    routes = {
        "BILLING": "handle_billing",
        "TECHNICAL": "handle_technical",
        "ACCOUNT": "handle_account",
        "GENERAL": "handle_general"
    }
    
    return routes.get(category, "handle_general")


# === HANDLER NODES ===

def handle_billing(state: TicketState) -> dict:
    """Handle billing-related tickets.
    
    Specializes in: payments, invoices, refunds, subscription issues.
    Uses a billing-focused prompt that knows about refund policies.
    """
    ticket = state["ticket_text"]
    
    prompt = f"""You are a billing support specialist. Help with this issue:
    
    {ticket}
    
    Be helpful and mention our refund policy if relevant.
    Keep response concise (2-3 sentences)."""
    
    response = llm.invoke(prompt)
    print("💳 Billing handler responded")
    
    return {"response": response.content, "needs_human": False}


def handle_technical(state: TicketState) -> dict:
    """Handle technical support tickets.
    
    Specializes in: bugs, errors, how-to questions, troubleshooting.
    Provides clear, step-by-step guidance.
    """
    ticket = state["ticket_text"]
    
    prompt = f"""You are a technical support specialist. Help with this issue:
    
    {ticket}
    
    Provide clear troubleshooting steps.
    Keep response concise (2-3 sentences)."""
    
    response = llm.invoke(prompt)
    print("🔧 Technical handler responded")
    
    return {"response": response.content, "needs_human": False}


def handle_account(state: TicketState) -> dict:
    """Handle account-related tickets.
    
    Specializes in: login issues, password reset, profile changes.
    Prioritizes security in responses.
    """
    ticket = state["ticket_text"]
    
    prompt = f"""You are an account support specialist. Help with this issue:
    
    {ticket}
    
    Prioritize security and verification.
    Keep response concise (2-3 sentences)."""
    
    response = llm.invoke(prompt)
    print("👤 Account handler responded")
    
    return {"response": response.content, "needs_human": False}


def handle_general(state: TicketState) -> dict:
    """Handle general inquiries that don't fit other categories."""
    ticket = state["ticket_text"]
    
    prompt = f"""You are a friendly support agent. Help with this inquiry:
    
    {ticket}
    
    Be warm and helpful.
    Keep response concise (2-3 sentences)."""
    
    response = llm.invoke(prompt)
    print("📧 General handler responded")
    
    return {"response": response.content, "needs_human": False}


def escalate_ticket(state: TicketState) -> dict:
    """Escalate high-priority tickets to human agents.
    
    This node doesn't try to solve the problem—it acknowledges
    the urgency and promises human follow-up.
    """
    print("🚨 Escalating to human agent")
    
    return {
        "response": "This ticket has been escalated to a senior support agent who will contact you within 1 hour.",
        "needs_human": True
    }


# === GRAPH BUILDER ===

def create_router_graph():
    """Build the ticket routing graph.
    
    The flow:
    1. classify - Analyze the ticket
    2. route_by_category - Decide which handler (conditional edge)
    3. One of five handlers runs
    4. END
    """
    graph = StateGraph(TicketState)
    
    # Add all our nodes
    graph.add_node("classify", classify_ticket)
    graph.add_node("handle_billing", handle_billing)
    graph.add_node("handle_technical", handle_technical)
    graph.add_node("handle_account", handle_account)
    graph.add_node("handle_general", handle_general)
    graph.add_node("escalate", escalate_ticket)
    
    # Start at classification
    graph.set_entry_point("classify")
    
    # After classification, route to the appropriate handler
    # This is the key part - 5-way conditional branching!
    graph.add_conditional_edges(
        "classify",
        route_by_category,
        {
            "handle_billing": "handle_billing",
            "handle_technical": "handle_technical",
            "handle_account": "handle_account",
            "handle_general": "handle_general",
            "escalate": "escalate"
        }
    )
    
    # All handlers lead to END (no loops in this graph)
    graph.add_edge("handle_billing", END)
    graph.add_edge("handle_technical", END)
    graph.add_edge("handle_account", END)
    graph.add_edge("handle_general", END)
    graph.add_edge("escalate", END)
    
    return graph.compile()


# === MAIN ===

def main():
    """Test the ticket router with different types of tickets."""
    app = create_router_graph()
    
    test_tickets = [
        "I was charged twice for my subscription last month!",
        "How do I reset my password?",
        "The app crashes whenever I try to upload a photo",
        "What are your business hours?",
        "THIS IS OUTRAGEOUS! Your service has been down for 3 hours!"
    ]
    
    print("=" * 60)
    print("🎫 Support Ticket Router")
    print("=" * 60)
    
    for ticket in test_tickets:
        print(f"\n📩 Ticket: {ticket[:50]}...")
        print("-" * 40)
        
        result = app.invoke({
            "ticket_text": ticket,
            "category": "",
            "priority": "",
            "response": "",
            "needs_human": False
        })
        
        print(f"📤 Response: {result['response'][:100]}...")
        if result["needs_human"]:
            print("⚠️  Escalated to human")


if __name__ == "__main__":
    main()


In [ ]:
# From: content_moderator.py

# From: Building AI Agents, Chapter 14, Section 14.6
# File: content_moderator.py

"""Content moderation with sequential decision gates.

Demonstrates Pattern 2: Chained Decisions
- Safety check → Topic check → Quality check
- Each gate can reject or pass to next
- Fail fast on safety violations
"""

import os
from typing import TypedDict
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


# === STATE ===

class ModerationState(TypedDict):
    content: str              # Content to moderate
    is_safe: bool             # Passes safety check?
    is_on_topic: bool         # Relevant to platform?
    quality_score: int        # Content quality (1-10)
    decision: str             # Final decision
    reason: str               # Explanation for the decision


# === GATE 1: SAFETY CHECK ===

def check_safety(state: ModerationState) -> dict:
    """First gate: Check for harmful content.
    
    This runs FIRST because there's no point checking topic or quality
    if the content is unsafe. We fail fast on safety violations.
    """
    content = state["content"]
    
    prompt = f"""Is this content safe and appropriate? Check for:
    - Hate speech or discrimination
    - Violence or threats
    - Adult content
    - Spam or scams
    
    Content: {content}
    
    Respond with only: SAFE or UNSAFE"""
    
    response = llm.invoke(prompt)
    is_safe = "SAFE" in response.content.upper() and "UNSAFE" not in response.content.upper()
    
    print(f"🛡️ Safety check: {'PASS' if is_safe else 'FAIL'}")
    
    return {"is_safe": is_safe}


def route_after_safety(state: ModerationState) -> str:
    """Route based on safety check result."""
    if state["is_safe"]:
        return "check_topic"      # Continue to next gate
    else:
        return "reject_unsafe"    # Stop here, reject immediately


# === GATE 2: TOPIC RELEVANCE ===

def check_topic(state: ModerationState) -> dict:
    """Second gate: Check if content is on-topic.
    
    We only reach here if safety passed. Now we check if
    the content belongs on our technology forum.
    """
    content = state["content"]
    
    prompt = f"""Is this content relevant to a technology discussion forum?
    
    Content: {content}
    
    Respond with only: ON_TOPIC or OFF_TOPIC"""
    
    response = llm.invoke(prompt)
    is_on_topic = "ON_TOPIC" in response.content.upper()
    
    print(f"🎯 Topic check: {'PASS' if is_on_topic else 'FAIL'}")
    
    return {"is_on_topic": is_on_topic}


def route_after_topic(state: ModerationState) -> str:
    """Route based on topic relevance."""
    if state["is_on_topic"]:
        return "check_quality"     # Continue to final gate
    else:
        return "reject_off_topic"  # Wrong forum


# === GATE 3: QUALITY ASSESSMENT ===

def check_quality(state: ModerationState) -> dict:
    """Third gate: Assess content quality.
    
    Safe, on-topic content still needs to meet quality standards.
    We use a 1-10 score for nuanced decisions:
    - 7-10: Approve
    - 4-6: Approve with suggestions
    - 1-3: Reject for low quality
    """
    content = state["content"]
    
    prompt = f"""Rate this content's quality from 1-10 based on:
    - Clarity and coherence
    - Usefulness to others
    - Effort and thoughtfulness
    
    Content: {content}
    
    Respond with only a number 1-10."""
    
    response = llm.invoke(prompt)
    
    # Parse the score with fallback
    try:
        score = int(''.join(filter(str.isdigit, response.content)))
        score = max(1, min(10, score))  # Clamp to valid range
    except:
        score = 5  # Default if parsing fails
    
    print(f"⭐ Quality score: {score}/10")
    
    return {"quality_score": score}


def route_after_quality(state: ModerationState) -> str:
    """Route based on quality score."""
    score = state["quality_score"]
    
    if score >= 7:
        return "approve"
    elif score >= 4:
        return "approve_with_note"
    else:
        return "reject_low_quality"


# === TERMINAL NODES ===

def approve(state: ModerationState) -> dict:
    """Approve high-quality content."""
    print("✅ Content approved!")
    return {
        "decision": "APPROVED",
        "reason": "Content meets all quality standards."
    }


def approve_with_note(state: ModerationState) -> dict:
    """Approve but suggest improvements."""
    print("✅ Content approved with suggestions")
    return {
        "decision": "APPROVED_WITH_SUGGESTIONS",
        "reason": f"Content approved. Quality: {state['quality_score']}/10. Consider adding more detail."
    }


def reject_unsafe(state: ModerationState) -> dict:
    """Reject content that failed safety check."""
    print("❌ Rejected: Safety violation")
    return {
        "decision": "REJECTED",
        "reason": "Content violates community safety guidelines."
    }


def reject_off_topic(state: ModerationState) -> dict:
    """Reject content that's not relevant."""
    print("❌ Rejected: Off-topic")
    return {
        "decision": "REJECTED",
        "reason": "Content is not relevant to this forum."
    }


def reject_low_quality(state: ModerationState) -> dict:
    """Reject content that failed quality check."""
    print("❌ Rejected: Low quality")
    return {
        "decision": "REJECTED",
        "reason": "Content does not meet quality standards. Please add more detail."
    }


# === GRAPH BUILDER ===

def create_moderation_graph():
    """Build the moderation pipeline.
    
    Visual flow:
    safety → (pass) → topic → (pass) → quality → approve/reject
              ↓                ↓                      
           reject           reject
    """
    graph = StateGraph(ModerationState)
    
    # Add all nodes
    graph.add_node("check_safety", check_safety)
    graph.add_node("check_topic", check_topic)
    graph.add_node("check_quality", check_quality)
    graph.add_node("approve", approve)
    graph.add_node("approve_with_note", approve_with_note)
    graph.add_node("reject_unsafe", reject_unsafe)
    graph.add_node("reject_off_topic", reject_off_topic)
    graph.add_node("reject_low_quality", reject_low_quality)
    
    # Start with safety
    graph.set_entry_point("check_safety")
    
    # Chain the decisions - each gate leads to the next or to rejection
    graph.add_conditional_edges(
        "check_safety",
        route_after_safety,
        {"check_topic": "check_topic", "reject_unsafe": "reject_unsafe"}
    )
    
    graph.add_conditional_edges(
        "check_topic",
        route_after_topic,
        {"check_quality": "check_quality", "reject_off_topic": "reject_off_topic"}
    )
    
    graph.add_conditional_edges(
        "check_quality",
        route_after_quality,
        {
            "approve": "approve",
            "approve_with_note": "approve_with_note",
            "reject_low_quality": "reject_low_quality"
        }
    )
    
    # All terminal nodes go to END
    for node in ["approve", "approve_with_note", "reject_unsafe", 
                 "reject_off_topic", "reject_low_quality"]:
        graph.add_edge(node, END)
    
    return graph.compile()


# === MAIN ===

def main():
    """Test the moderation pipeline with various content."""
    app = create_moderation_graph()
    
    test_posts = [
        "Here's my detailed guide on setting up Docker containers for Python development...",
        "Check out this awesome new JavaScript framework I found!",
        "HATE HATE HATE everyone who uses tabs instead of spaces!!!",
        "Anyone want to buy cheap watches? Click here: scam.com",
        "hi",
        "What's your favorite recipe for chocolate chip cookies?",
    ]
    
    print("=" * 60)
    print("🔍 Content Moderation Pipeline")
    print("=" * 60)
    
    for post in test_posts:
        print(f"\n📝 Post: {post[:50]}...")
        print("-" * 40)
        
        result = app.invoke({
            "content": post,
            "is_safe": False,
            "is_on_topic": False,
            "quality_score": 0,
            "decision": "",
            "reason": ""
        })
        
        print(f"📋 Decision: {result['decision']}")
        print(f"📋 Reason: {result['reason']}")


if __name__ == "__main__":
    main()


---
### Section 14.6 Exercises

### Exercise 14.6.1: Email Classifier

Build a graph that classifies incoming emails and routes them to specialized handlers:
- URGENT → Generate quick acknowledgment
- MEETING → Extract date, time, participants  
- NEWSLETTER → Archive it
- PERSONAL → Flag for personal review
- SPAM → Delete it

Your graph should have:
- One classification node
- Five different handler nodes
- A routing function that maps categories to handlers

In [ ]:
# Your code here


### Exercise 14.6.2: Multi-Stage Interview

Create an interview bot with three stages:
- Stage 1: Basic info (name, background)
- Stage 2: Technical questions (different paths for engineer vs designer)
- Stage 3: Behavioral questions

Requirements:
- Only advance when current stage is complete
- Engineers and designers get different technical questions
- End with a summary of the interview

In [ ]:
# Your code here


### Exercise 14.6.3: Retry with Backoff

Enhance a research assistant to handle poor-quality results:
- If search quality is LOW, retry with a modified query
- Track retries per search (max 2 retries)
- If still low after retries, move on to next search
- Add `retry_count` and `current_quality` to state

In [ ]:
# Your code here


---
## Section 14.7: Debugging LangGraph flows

In [ ]:
# From: debug_utils.py

# From: Building AI Agents, Chapter 14, Section 14.7
# File: debug_utils.py

"""Debugging utilities for LangGraph applications.

Provides decorators to add debug output to nodes and routing functions
without cluttering the main logic.
"""


def debug_node(name: str):
    """Decorator that adds debug output to any node function.
    
    Usage:
        @debug_node("my_node")
        def my_node(state: MyState) -> dict:
            ...
    """
    def decorator(func):
        def wrapper(state):
            # Print entry
            print(f"\n{'='*50}")
            print(f"🔵 ENTERING: {name}")
            print(f"{'='*50}")
            
            # Print incoming state
            print(f"📥 State received:")
            for key, value in state.items():
                str_val = str(value)[:60] + "..." if len(str(value)) > 60 else str(value)
                print(f"   {key}: {str_val}")
            
            # Call the actual function
            result = func(state)
            
            # Print outgoing updates
            print(f"\n📤 Returning updates:")
            if result:
                for key, value in result.items():
                    str_val = str(value)[:60] + "..." if len(str(value)) > 60 else str(value)
                    print(f"   {key}: {str_val}")
            else:
                print("   (no updates)")
            
            print(f"{'='*50}\n")
            
            return result
        return wrapper
    return decorator


def debug_router(name: str):
    """Decorator that adds debug output to routing functions.
    
    Usage:
        @debug_router("my_router")
        def my_router(state: MyState) -> str:
            ...
    """
    def decorator(func):
        def wrapper(state):
            result = func(state)
            print(f"🔀 ROUTER '{name}' decided: {result}")
            return result
        return wrapper
    return decorator


# Example usage
if __name__ == "__main__":
    from typing import TypedDict
    
    class ExampleState(TypedDict):
        message: str
        processed: bool
    
    @debug_node("example_node")
    def example_node(state: ExampleState) -> dict:
        return {"processed": True}
    
    @debug_router("example_router")
    def example_router(state: ExampleState) -> str:
        return "next" if state.get("processed") else "process"
    
    # Test
    test_state = {"message": "Hello", "processed": False}
    result = example_node(test_state)
    decision = example_router(test_state)


In [ ]:
# From: state_tracker.py

# From: Building AI Agents, Chapter 14, Section 14.7
# File: state_tracker.py

"""Track state changes through graph execution.

Captures the full state at each node for later analysis.
Useful for debugging complex state transformations.
"""

import copy
from datetime import datetime


class StateTracker:
    """Captures state at each node for later analysis."""
    
    def __init__(self):
        self.history = []
    
    def capture(self, node_name: str, state: dict, updates: dict = None):
        """Record state at a point in execution.
        
        Args:
            node_name: Name of the current node
            state: The state dictionary before updates
            updates: The updates being returned (optional)
        """
        snapshot = {
            "timestamp": datetime.now().isoformat(),
            "node": node_name,
            "state_before": copy.deepcopy(dict(state)),
            "updates": copy.deepcopy(updates) if updates else None
        }
        self.history.append(snapshot)
    
    def print_history(self):
        """Print the execution history."""
        print("\n" + "=" * 60)
        print("📜 EXECUTION HISTORY")
        print("=" * 60)
        
        for i, snapshot in enumerate(self.history):
            print(f"\n--- Step {i + 1}: {snapshot['node']} ---")
            print(f"Time: {snapshot['timestamp']}")
            
            if snapshot['updates']:
                print("Updates made:")
                for key, value in snapshot['updates'].items():
                    print(f"  {key}: {value}")
    
    def find_changes(self, field: str):
        """Track how a specific field changed over time.
        
        Args:
            field: The state field to track
        """
        print(f"\n📊 History of '{field}':")
        
        for snapshot in self.history:
            value = snapshot['state_before'].get(field, '<not set>')
            update = snapshot['updates'].get(field, '<no change>') if snapshot['updates'] else '<no change>'
            print(f"  {snapshot['node']}: {value} → {update}")
    
    def clear(self):
        """Reset the history."""
        self.history = []


# Global tracker instance for easy import
tracker = StateTracker()


# Example usage
if __name__ == "__main__":
    # Simulate some node executions
    tracker.capture("node_1", {"count": 0, "status": "starting"}, {"count": 1})
    tracker.capture("node_2", {"count": 1, "status": "starting"}, {"status": "processing"})
    tracker.capture("node_3", {"count": 1, "status": "processing"}, {"count": 2, "status": "done"})
    
    # Show the history
    tracker.print_history()
    
    # Track a specific field
    tracker.find_changes("count")
    tracker.find_changes("status")


In [ ]:
# From: step_executor.py

# From: Building AI Agents, Chapter 14, Section 14.7
# File: step_executor.py

"""Execute a graph step by step for debugging.

Allows you to pause between nodes, inspect state,
and understand the execution flow interactively.
"""


def step_through(app, initial_state: dict):
    """Execute graph step by step, pausing between nodes.
    
    This lets you inspect state after each node.
    
    Args:
        app: A compiled LangGraph application
        initial_state: The initial state dictionary
        
    Returns:
        The final state after execution, or None if stopped early
    """
    print("\n🐛 Step-Through Debugger")
    print("=" * 50)
    print("Commands: [enter]=next, 's'=show state, 'q'=quit")
    print("=" * 50)
    
    # Get stream of execution steps
    step_count = 0
    
    for event in app.stream(initial_state):
        step_count += 1
        
        # event is a dict with the node name as key
        for node_name, node_output in event.items():
            print(f"\n--- Step {step_count}: {node_name} completed ---")
            
            # Show what this node returned
            if node_output:
                print("Output:")
                for key, value in node_output.items():
                    str_val = str(value)[:80]
                    print(f"  {key}: {str_val}")
        
        # Interactive prompt
        cmd = input("\n> ").strip().lower()
        
        if cmd == 'q':
            print("Stopped by user")
            return None
        elif cmd == 's':
            print("\nFull state would be shown here")
            # Note: Getting full state mid-stream requires checkpointing
            # which we'll cover in Chapter 15
    
    print(f"\n✅ Execution complete ({step_count} steps)")
    return event


# Example usage
if __name__ == "__main__":
    print("This module provides the step_through() function.")
    print("Usage:")
    print("  from step_executor import step_through")
    print("  app = create_my_graph()")
    print("  final_state = step_through(app, initial_state)")


In [ ]:
# From: debug_template.py

# From: Building AI Agents, Chapter 14, Section 14.7
# File: debug_template.py

"""Template for debug-ready LangGraph applications.

Use this as a starting point for new graphs that include
debugging from the start. Set DEBUG = False for production.
"""

import os
from typing import TypedDict, Annotated
from operator import add
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

load_dotenv()
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Debug flag - set to False in production
DEBUG = True


def debug_print(*args, **kwargs):
    """Print only if DEBUG is True."""
    if DEBUG:
        print(*args, **kwargs)


# === STATE ===

class MyState(TypedDict):
    input: str
    output: str
    step_count: int  # Track iterations


# === NODES (with debug output) ===

def my_node(state: MyState) -> dict:
    debug_print(f"\n🔵 my_node - Entered")
    debug_print(f"   Input: {state.get('input', 'N/A')[:50]}")
    
    # ... your logic here ...
    result = "processed"
    
    updates = {
        "output": result,
        "step_count": state.get("step_count", 0) + 1
    }
    
    debug_print(f"   Output: {result[:50]}")
    debug_print(f"   Step: {updates['step_count']}")
    
    return updates


# === ROUTING (with debug output) ===

def route_decision(state: MyState) -> str:
    decision = "end"  # Your logic here
    
    debug_print(f"🔀 route_decision: {decision}")
    
    return decision


# === GRAPH ===

def create_graph():
    graph = StateGraph(MyState)
    
    graph.add_node("my_node", my_node)
    graph.set_entry_point("my_node")
    
    graph.add_conditional_edges(
        "my_node",
        route_decision,
        {"continue": "my_node", "end": END}
    )
    
    return graph.compile()


# === MAIN ===

def main():
    app = create_graph()
    
    # Print graph structure
    if DEBUG:
        print("\n📊 Graph Structure:")
        print(app.get_graph().draw_mermaid())
    
    initial_state = {
        "input": "test input",
        "output": "",
        "step_count": 0
    }
    
    debug_print("\n🚀 Starting execution...")
    result = app.invoke(initial_state)
    
    debug_print(f"\n✅ Complete! Steps: {result['step_count']}")
    print(f"\nFinal output: {result['output']}")


if __name__ == "__main__":
    main()


In [ ]:
# From: document_analyzer_challenge.py

# From: Building AI Agents, Chapter 14 Challenge Project
# Save as: document_analyzer_challenge.py
# Challenge: Build a Multi-Stage Document Analyzer

"""
CHAPTER 14 CHALLENGE: Multi-Stage Document Analyzer

Build a sophisticated document analysis agent that demonstrates everything
you learned in Chapter 14:
- State design with TypedDict and Annotated[list, add]
- Multiple nodes (at least 6)
- Multi-way branching (4+ document types)
- Quality-check loops with iteration limits
- Debugging support

REQUIREMENTS:
1. Classify documents into 4+ types (technical, business, legal, academic)
2. Route to specialized extraction nodes based on type
3. Evaluate extraction quality and retry if needed (max 2 retries)
4. Accumulate extracted information using Annotated[list, add]
5. Include debug output for tracing execution

YOUR TASKS:
1. Complete the State definition
2. Implement all node functions
3. Create the routing function
4. Build and compile the graph
5. Test with the sample documents provided

Good luck! 🚀
"""

import os
from typing import TypedDict, Annotated, Literal
from operator import add
from dotenv import load_dotenv

load_dotenv()

# Uncomment when ready to use LLM
# from langchain_openai import ChatOpenAI
# from langgraph.graph import StateGraph, START, END

# =============================================================================
# DEBUG FLAG - Set to True to see execution trace
# =============================================================================
DEBUG = True

def debug_print(*args, **kwargs):
    """Print only when DEBUG is True."""
    if DEBUG:
        print(*args, **kwargs)


# =============================================================================
# STATE DEFINITION
# =============================================================================

class DocumentState(TypedDict):
    """State for the document analyzer.
    
    TODO: Complete this state definition with:
    - document: str - The input document text
    - doc_type: str - Classification result (technical/business/legal/academic)
    - extracted_info: Annotated[list, add] - Accumulated extractions
    - quality_score: float - Quality of extraction (0.0 to 1.0)
    - iteration_count: int - Number of extraction attempts
    - max_iterations: int - Maximum allowed attempts
    - final_summary: str - Final analysis summary
    """
    # TODO: Add your state fields here
    pass


# =============================================================================
# NODE FUNCTIONS
# =============================================================================

def classify_document(state: DocumentState) -> dict:
    """Classify the document into one of 4 types.
    
    Types:
    - technical: Research papers, technical docs, API documentation
    - business: Reports, memos, financial documents
    - legal: Contracts, agreements, legal notices
    - academic: Essays, thesis, scholarly articles
    
    TODO: Implement classification logic
    - Use keywords or LLM to classify
    - Return {"doc_type": "technical|business|legal|academic"}
    """
    debug_print(f"\n{'='*50}")
    debug_print("🔵 ENTERING: classify_document")
    debug_print(f"{'='*50}")
    
    document = state.get("document", "")
    
    # TODO: Implement classification
    # Hint: Look for keywords like "abstract", "whereas", "quarterly", "methodology"
    
    doc_type = "technical"  # Placeholder
    
    debug_print(f"📄 Classified as: {doc_type}")
    return {"doc_type": doc_type}


def extract_technical(state: DocumentState) -> dict:
    """Extract information from technical documents.
    
    Extract:
    - Methods/approaches used
    - Key findings
    - Technologies mentioned
    
    TODO: Implement extraction logic
    - Return {"extracted_info": [list of extracted items]}
    - Increment iteration_count
    """
    debug_print(f"\n{'='*50}")
    debug_print("🔵 ENTERING: extract_technical")
    debug_print(f"{'='*50}")
    
    # TODO: Implement technical extraction
    
    return {
        "extracted_info": ["[Technical extraction placeholder]"],
        "iteration_count": state.get("iteration_count", 0) + 1
    }


def extract_business(state: DocumentState) -> dict:
    """Extract information from business documents.
    
    Extract:
    - Key metrics and numbers
    - Decisions made
    - Action items
    
    TODO: Implement extraction logic
    """
    debug_print(f"\n{'='*50}")
    debug_print("🔵 ENTERING: extract_business")
    debug_print(f"{'='*50}")
    
    # TODO: Implement business extraction
    
    return {
        "extracted_info": ["[Business extraction placeholder]"],
        "iteration_count": state.get("iteration_count", 0) + 1
    }


def extract_legal(state: DocumentState) -> dict:
    """Extract information from legal documents.
    
    Extract:
    - Parties involved
    - Key obligations
    - Important dates
    
    TODO: Implement extraction logic
    """
    debug_print(f"\n{'='*50}")
    debug_print("🔵 ENTERING: extract_legal")
    debug_print(f"{'='*50}")
    
    # TODO: Implement legal extraction
    
    return {
        "extracted_info": ["[Legal extraction placeholder]"],
        "iteration_count": state.get("iteration_count", 0) + 1
    }


def extract_academic(state: DocumentState) -> dict:
    """Extract information from academic documents.
    
    Extract:
    - Main thesis/argument
    - Methodology
    - Conclusions
    
    TODO: Implement extraction logic
    """
    debug_print(f"\n{'='*50}")
    debug_print("🔵 ENTERING: extract_academic")
    debug_print(f"{'='*50}")
    
    # TODO: Implement academic extraction
    
    return {
        "extracted_info": ["[Academic extraction placeholder]"],
        "iteration_count": state.get("iteration_count", 0) + 1
    }


def evaluate_quality(state: DocumentState) -> dict:
    """Evaluate the quality of extraction.
    
    TODO: Implement quality evaluation
    - Check if extracted_info is meaningful
    - Return {"quality_score": 0.0 to 1.0}
    
    Quality criteria:
    - At least 3 items extracted
    - Items are not placeholders
    - Items are relevant to doc_type
    """
    debug_print(f"\n{'='*50}")
    debug_print("🔵 ENTERING: evaluate_quality")
    debug_print(f"{'='*50}")
    
    extracted = state.get("extracted_info", [])
    
    # TODO: Implement quality scoring
    # Placeholder: simple length-based score
    quality_score = min(len(extracted) / 5, 1.0)
    
    debug_print(f"📊 Quality score: {quality_score}")
    debug_print(f"📊 Iteration: {state.get('iteration_count', 0)}")
    
    return {"quality_score": quality_score}


def generate_summary(state: DocumentState) -> dict:
    """Generate final summary of the analysis.
    
    TODO: Implement summary generation
    - Combine all extracted information
    - Create a coherent summary
    - Return {"final_summary": "..."}
    """
    debug_print(f"\n{'='*50}")
    debug_print("🔵 ENTERING: generate_summary")
    debug_print(f"{'='*50}")
    
    doc_type = state.get("doc_type", "unknown")
    extracted = state.get("extracted_info", [])
    
    # TODO: Generate a meaningful summary
    summary = f"Analysis of {doc_type} document. Found {len(extracted)} items."
    
    debug_print(f"📝 Summary generated")
    return {"final_summary": summary}


# =============================================================================
# ROUTING FUNCTIONS
# =============================================================================

def route_by_doc_type(state: DocumentState) -> Literal["extract_technical", "extract_business", "extract_legal", "extract_academic"]:
    """Route to appropriate extraction node based on document type.
    
    TODO: Implement routing logic
    - Read doc_type from state
    - Return the appropriate node name
    """
    debug_print(f"\n🔀 ROUTING: route_by_doc_type")
    
    doc_type = state.get("doc_type", "technical")
    
    # TODO: Implement routing
    route_map = {
        "technical": "extract_technical",
        "business": "extract_business",
        "legal": "extract_legal",
        "academic": "extract_academic"
    }
    
    destination = route_map.get(doc_type, "extract_technical")
    debug_print(f"   → Going to: {destination}")
    
    return destination


def route_quality_check(state: DocumentState) -> Literal["generate_summary", "retry_extraction"]:
    """Decide whether to retry extraction or proceed to summary.
    
    TODO: Implement quality check routing
    - If quality_score >= 0.7, go to generate_summary
    - If iteration_count >= max_iterations, go to generate_summary (give up)
    - Otherwise, go to retry_extraction
    """
    debug_print(f"\n🔀 ROUTING: route_quality_check")
    
    quality = state.get("quality_score", 0)
    iterations = state.get("iteration_count", 0)
    max_iter = state.get("max_iterations", 2)
    
    # TODO: Implement routing logic
    if quality >= 0.7:
        debug_print(f"   ✅ Quality sufficient ({quality}), proceeding to summary")
        return "generate_summary"
    elif iterations >= max_iter:
        debug_print(f"   ⚠️ Max iterations reached ({iterations}), proceeding anyway")
        return "generate_summary"
    else:
        debug_print(f"   🔄 Quality low ({quality}), retrying extraction")
        return "retry_extraction"


# =============================================================================
# GRAPH CONSTRUCTION
# =============================================================================

def build_document_analyzer():
    """Build the document analyzer graph.
    
    TODO: Implement the graph structure
    
    Graph structure:
    START → classify_document → [route_by_doc_type] → extract_* → evaluate_quality
                                                                        ↓
                                                        [route_quality_check]
                                                           ↓           ↓
                                              generate_summary    retry (loop back)
                                                      ↓
                                                     END
    
    Hints:
    1. Create StateGraph with DocumentState
    2. Add all nodes
    3. Add edge from START to classify_document
    4. Add conditional edges for routing
    5. Add edge from generate_summary to END
    6. Handle retry loop (goes back to appropriate extract_* node)
    """
    
    # TODO: Build your graph here
    # 
    # graph = StateGraph(DocumentState)
    # 
    # # Add nodes
    # graph.add_node("classify_document", classify_document)
    # graph.add_node("extract_technical", extract_technical)
    # ... add more nodes ...
    # 
    # # Add edges
    # graph.add_edge(START, "classify_document")
    # graph.add_conditional_edges(
    #     "classify_document",
    #     route_by_doc_type,
    #     {...}
    # )
    # ... add more edges ...
    # 
    # return graph.compile()
    
    print("TODO: Implement build_document_analyzer()")
    return None


# =============================================================================
# TEST DOCUMENTS
# =============================================================================

SAMPLE_DOCUMENTS = {
    "technical": """
    Abstract: This paper presents a novel approach to natural language processing
    using transformer architectures. We implement a BERT-based model with custom
    attention mechanisms. Our methodology involves fine-tuning on domain-specific
    data. Key findings show 15% improvement in accuracy. Technologies used include
    PyTorch, Hugging Face Transformers, and CUDA for GPU acceleration.
    """,
    
    "business": """
    Q3 2024 Performance Report
    
    Revenue increased 23% year-over-year to $4.2M. The board has decided to
    expand into European markets. Action items: 1) Hire regional sales manager
    by Dec 1, 2) Complete compliance audit by Nov 15, 3) Launch marketing
    campaign in January. Customer acquisition cost decreased to $45.
    """,
    
    "legal": """
    SERVICE AGREEMENT
    
    This Agreement is entered into between ABC Corporation ("Provider") and
    XYZ Inc. ("Client") effective January 1, 2025. Provider agrees to deliver
    consulting services as described in Exhibit A. Client shall pay $10,000
    monthly. This agreement shall terminate on December 31, 2025. Either party
    may terminate with 30 days written notice.
    """,
    
    "academic": """
    Introduction: This thesis examines the impact of social media on political
    discourse. The central argument posits that algorithmic curation creates
    echo chambers. Our methodology combines quantitative analysis of 10,000
    posts with qualitative interviews of 50 participants. We conclude that
    platform design significantly influences information diversity. Future
    research should explore intervention strategies.
    """
}


# =============================================================================
# MAIN - Test your implementation
# =============================================================================

def main():
    """Test the document analyzer with sample documents."""
    
    print("=" * 60)
    print("📄 DOCUMENT ANALYZER CHALLENGE")
    print("=" * 60)
    
    # Build the graph
    analyzer = build_document_analyzer()
    
    if analyzer is None:
        print("\n❌ Graph not implemented yet!")
        print("Complete the TODO items and try again.")
        return
    
    # Test with each document type
    for doc_type, document in SAMPLE_DOCUMENTS.items():
        print(f"\n{'='*60}")
        print(f"📄 Testing {doc_type.upper()} document")
        print(f"{'='*60}")
        
        initial_state = {
            "document": document,
            "doc_type": "",
            "extracted_info": [],
            "quality_score": 0.0,
            "iteration_count": 0,
            "max_iterations": 2,
            "final_summary": ""
        }
        
        try:
            result = analyzer.invoke(initial_state)
            
            print(f"\n✅ RESULTS:")
            print(f"   Document type: {result.get('doc_type')}")
            print(f"   Items extracted: {len(result.get('extracted_info', []))}")
            print(f"   Quality score: {result.get('quality_score')}")
            print(f"   Iterations: {result.get('iteration_count')}")
            print(f"   Summary: {result.get('final_summary', '')[:100]}...")
            
        except Exception as e:
            print(f"\n❌ Error: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"\n{'='*60}")
    print("🏁 Challenge complete!")
    print("=" * 60)


if __name__ == "__main__":
    main()

---
### Section 14.7 Exercises

### Exercise 14.7.1: Add Debugging to the Ticket Router

Take the ticket router from section 14.6 and add:
- Debug output for every node
- State tracking
- A loop counter safety valve
- Graph visualization at startup

In [ ]:
# Your code here


### Exercise 14.7.2: Find the Bug

Here's a buggy graph with 3 bugs. Use debugging techniques to find and fix:
1. List not accumulating (missing `Annotated[list, add]`)
2. KeyError on state access (missing `.get()` with default)
3. Routing mismatch (return values don't match mapping keys)

In [ ]:
# Your code here


### Exercise 14.7.3: Build a Debug Dashboard

Create a function that produces a summary report:
- Total nodes visited
- Time spent
- State changes for each field
- Fields that never changed
- Routing decisions made

In [ ]:
# Your code here


---
## Next Steps

- Check your answers in **chapter_14_langgraph_intro_solutions.ipynb**
- Proceed to **Chapter 15**